In [1]:
import pandas as pd
import string
pd.set_option('max_colwidth', 800)
# import numpy as np

#### Observations 
- Each data set consists of 3 files, the dataset, the metadata, and a text file. 
- The dataset is formatted with with geographical identifiers (zip code) as columns, and each attribute as its own column. 
- Column names are coded. But they are not unique from one data table to another => need a way to uniquely identify column names.
- The first row of each data table contains an explanation of the variable => this part needs to be formatted and removed when the table is in use for modeling or analysis. 
- Data tables contain data of type: estimates (count or percentage), and margin of error (MOE) of each estimate. At times, choices will be made regarding variables that are useful for our purposes, e.g. as of now, we are not using variables pertaining to MOEs, percent allocated. 


In [89]:
def load_data_df(bucket, table_code):
    """loading csv file downloaded from US Census"""
    return pd.read_csv('{}/ACS_17_5YR_{}_with_ann.csv'.format(bucket, table_code),
                       na_values=['(X)'],
                       dtype = 'unicode')

def load_metadata_df(bucket, table_code):
    """loading csv file downloaded from US Census"""
    return pd.read_csv('{}/ACS_17_5YR_{}_metadata.csv'.format(bucket, table_code),
                       na_values=['(X)'],
                       dtype = 'unicode')

def keep_cols(metadata_df, data_df, key_words_in_col):
    """
    return original data_df only with columns that contain key words in key_words_in_col, along with 5-digit zip
    """
    cols_to_keep = []
    
    #adding 5-digit zip code ID
    cols_to_keep.append('GEO.id2')
    
    for row in metadata_df.itertuples(index=False):
        has_key_word = [x for x in key_words_in_col if x in row[1].lower()]
        if any(has_key_word):
            cols_to_keep.append(row[0])
    
    #verifying that cols_to_keep are all in data_df 
    unexisting_cols = [c for c in cols_to_keep if c not in data_df.columns]
    if any(unexisting_cols):
        print("""Df doesn't have all the columns as indicated in metadata, these are {} \n
        Proceeding with keep_cols anyway, disregarding these columns""".format(unexisting_cols))
        cols_to_keep = [c for c in cols_to_keep if c not in unexisting_cols]
    
    return data_df[cols_to_keep]

def exclude_cols(metadata_df, data_df, key_words_in_col):
    """
    return original data_df only ***exluding*** columns that contain key words in key_words_in_col, along with 5-digit zip
    """
    cols_to_exclude= []
    
    for row in metadata_df.itertuples(index=False):
        has_key_word = [x for x in key_words_in_col if x in row[1].lower()]
        if any(has_key_word):
            cols_to_exclude.append(row[0])
            
    return data_df[data_df.columns.difference(cols_to_exclude)]

def drop_na_cols(df):
    return df.dropna(axis = 1)

def show_manhattan(df):
    """
    returns only rows containing manhattan zip codes
    for EDA purposes
    """
    manhattan_zipcodes = ['Id2', '10026', '10027', '10030', '10037', '10039', 
                      '10029', '10035', '10010', '10016', '10017', '10022',
                      '10012', '10013', '10014', '10004', '10005', '10006', '10007', '10038', '10280',
                      '10002', '10003', '10009', '10021', '10028', '10044', '10065', '10075', '10128',
                      '10023', '10024', '10025', '10031', '10032', '10033', '10034', '10040',
                      '10001', '10011', '10018', '10019', '10020', '10036']
    return df.loc[df['GEO.id2'].isin(manhattan_zipcodes)]

def replace_all(text, dic):
    """
    replace string by conventions set out by external dictionary with each key as the replaced, 
    and value as the replacee
    """
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def extract_variable_description_row(df):
    """returns a series"""
    return df.iloc[:1].transpose()[0]

def get_selected_attributes_names(df, dic):
    """returns a list"""
    unprocessed_variable_descriptions = extract_variable_description_row(df)
    return [replace_all(x.lower(), dic) for x in unprocessed_variable_descriptions]

In [75]:
#racial indicator

# {
#     'A': 'white_alone',
#     'B': 'black_alone',
#     'C': ''
# }

metadata_replacement_dict = { 
    "-": "_", 
    ";": "_", 
    " ": "_", 
    "_or_more": "+",
    "less_than_": "<",
    ":": "",
    "estimate": "est"
    }

### Sex by Age

In [82]:
sex_by_age_df = load_data_df('acs_sex_age_edu', 'S0101')
sex_metadata_df = load_metadata_df('acs_sex_age_edu', 'S0101')

In [83]:
sex_by_age_df.shape, sex_by_age_df.columns

((33121, 459),
 Index(['GEO.id', 'GEO.id2', 'GEO.display-label', 'HC01_EST_VC01',
        'HC01_MOE_VC01', 'HC02_EST_VC01', 'HC02_MOE_VC01', 'HC03_EST_VC01',
        'HC03_MOE_VC01', 'HC04_EST_VC01',
        ...
        'HC02_EST_VC45', 'HC02_MOE_VC45', 'HC03_EST_VC45', 'HC03_MOE_VC45',
        'HC04_EST_VC45', 'HC04_MOE_VC45', 'HC05_EST_VC45', 'HC05_MOE_VC45',
        'HC06_EST_VC45', 'HC06_MOE_VC45'],
       dtype='object', length=459))

In [84]:
sex_by_age_df = keep_cols(sex_metadata_df, sex_by_age_df, ['estimate'])
sex_by_age_df.shape

(33121, 229)

In [85]:
sex_by_age_df = drop_na_cols(sex_by_age_df)
sex_by_age_df.shape

(33121, 193)

In [86]:
sex_by_age_df = exclude_cols(sex_metadata_df, sex_by_age_df, ['allocated', 'selected age categories'])
sex_by_age_df.shape

(33121, 119)

In [7]:
sex_by_age_df.head(20)

,GEO.id2,HC01_EST_VC01,HC01_EST_VC03,HC01_EST_VC04,HC01_EST_VC05,HC01_EST_VC06,HC01_EST_VC07,HC01_EST_VC08,HC01_EST_VC09,HC01_EST_VC10,...,HC06_EST_VC11,HC06_EST_VC12,HC06_EST_VC13,HC06_EST_VC14,HC06_EST_VC15,HC06_EST_VC16,HC06_EST_VC17,HC06_EST_VC18,HC06_EST_VC19,HC06_EST_VC20
0,Id2,Total; Estimate; Total population,Total; Estimate; AGE - Under 5 years,Total; Estimate; AGE - 5 to 9 years,Total; Estimate; AGE - 10 to 14 years,Total; Estimate; AGE - 15 to 19 years,Total; Estimate; AGE - 20 to 24 years,Total; Estimate; AGE - 25 to 29 years,Total; Estimate; AGE - 30 to 34 years,Total; Estimate; AGE - 35 to 39 years,...,Percent Female; Estimate; AGE - 40 to 44 years,Percent Female; Estimate; AGE - 45 to 49 years,Percent Female; Estimate; AGE - 50 to 54 years,Percent Female; Estimate; AGE - 55 to 59 years,Percent Female; Estimate; AGE - 60 to 64 years,Percent Female; Estimate; AGE - 65 to 69 years,Percent Female; Estimate; AGE - 70 to 74 years,Percent Female; Estimate; AGE - 75 to 79 years,Percent Female; Estimate; AGE - 80 to 84 years,Percent Female; Estimate; AGE - 85 years and over
1,00601,17599,938,1046,1278,1250,1218,1148,1073,1121,...,6.1,6.5,7.3,6.9,6.8,5.4,4.6,3.0,2.2,2.4
2,00602,39209,1794,2320,2113,2706,2751,2537,2350,2715,...,5.2,7.2,7.7,7.4,7.4,5.1,5.0,2.4,2.2,2.1
3,00603,50135,2469,2778,3260,3443,3194,3264,3021,3296,...,6.1,6.2,7.0,6.0,7.2,7.0,5.0,3.5,2.9,2.0
4,00606,6304,312,304,420,479,443,302,329,279,...,7.7,6.0,8.6,4.5,9.7,6.6,4.2,2.2,1.1,3.6
5,00610,27590,1256,1346,1895,1939,1924,1624,1575,1638,...,6.3,7.5,7.2,7.0,6.1,5.7,5.8,4.0,2.5,1.9
6,00612,62566,3260,3077,4249,4397,4346,3621,3586,3550,...,6.6,6.5,7.0,5.7,6.3,7.1,5.0,4.5,3.0,2.9
7,00616,10687,469,417,405,678,901,784,574,661,...,6.5,10.0,5.7,5.6,6.7,5.9,6.6,3.0,4.2,2.7
8,00617,24508,1340,1401,1702,1686,1709,1607,1570,1719,...,7.3,6.7,5.9,5.4,5.7,5.5,4.9,2.9,2.1,2.9
9,00622,7405,224,296,489,477,428,243,352,382,...,5.7,4.1,9.2,8.8,5.4,5.8,8.6,6.1,3.8,4.6


In [90]:
get_selected_attributes_names(sex_by_age_df, metadata_replacement_dict)

['id2',
 'total__est__total_population',
 'total__est__age___under_5_years',
 'total__est__age___5_to_9_years',
 'total__est__age___10_to_14_years',
 'total__est__age___15_to_19_years',
 'total__est__age___20_to_24_years',
 'total__est__age___25_to_29_years',
 'total__est__age___30_to_34_years',
 'total__est__age___35_to_39_years',
 'total__est__age___40_to_44_years',
 'total__est__age___45_to_49_years',
 'total__est__age___50_to_54_years',
 'total__est__age___55_to_59_years',
 'total__est__age___60_to_64_years',
 'total__est__age___65_to_69_years',
 'total__est__age___70_to_74_years',
 'total__est__age___75_to_79_years',
 'total__est__age___80_to_84_years',
 'total__est__age___85_years_and_over',
 'total__est__summary_indicators___median_age_(years)',
 'total__est__summary_indicators___sex_ratio_(males_per_100_females)',
 'total__est__summary_indicators___age_dependency_ratio',
 'total__est__summary_indicators___old_age_dependency_ratio',
 'total__est__summary_indicators___child_depen

### Education

In [97]:
edu_bucket = 'edu_25+'
table_code = 'B15002'
edu_data_df = load_data_df(edu_bucket, table_code)
edu_metadata_df = load_metadata_df(edu_bucket, table_code)
print('Original df dimensions: {}'.format(edu_data_df.shape))

edu_data_df = keep_cols(edu_metadata_df, edu_data_df, ['estimate'])
print('After keeping only estimates cols df dimensions: {}'.format(edu_data_df.shape))

Original df dimensions: (33121, 73)
After keeping only estimates cols df dimensions: (33121, 36)


In [98]:
show_manhattan(edu_data_df)

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,...,HD01_VD26,HD01_VD27,HD01_VD28,HD01_VD29,HD01_VD30,HD01_VD31,HD01_VD32,HD01_VD33,HD01_VD34,HD01_VD35
0,Id2,Estimate; Total:,Estimate; Male:,Estimate; Male: - No schooling completed,Estimate; Male: - Nursery to 4th grade,Estimate; Male: - 5th and 6th grade,Estimate; Male: - 7th and 8th grade,Estimate; Male: - 9th grade,Estimate; Male: - 10th grade,Estimate; Male: - 11th grade,...,Estimate; Female: - 11th grade,"Estimate; Female: - 12th grade, no diploma",Estimate; Female: - High school graduate (includes equivalency),"Estimate; Female: - Some college, less than 1 year","Estimate; Female: - Some college, 1 or more years, no degree",Estimate; Female: - Associate's degree,Estimate; Female: - Bachelor's degree,Estimate; Female: - Master's degree,Estimate; Female: - Professional school degree,Estimate; Female: - Doctorate degree
2559,10001,17839,9693,44,0,92,32,73,114,42,...,29,70,898,147,625,220,3074,1779,604,167
2560,10002,60938,29079,1442,429,1717,1342,1465,855,752,...,460,1220,6605,675,2068,1492,6083,2767,644,353
2561,10003,39339,20095,37,87,40,18,25,109,146,...,72,102,790,239,1358,481,8896,4370,1445,929
2562,10004,2409,1079,18,0,0,0,0,0,0,...,0,0,21,14,60,28,754,315,81,42
2563,10005,6403,2909,0,0,0,0,0,1,1,...,0,0,170,0,69,192,1679,844,324,173
2564,10006,2657,1185,0,0,0,0,0,0,0,...,0,0,0,0,29,18,816,298,246,48
2565,10007,5336,3117,34,0,14,0,3,162,28,...,2,2,121,0,59,0,920,507,409,77
2566,10009,46930,21389,303,288,374,379,312,391,239,...,238,308,3801,641,2001,1245,8495,4343,1280,559
2567,10010,25360,13250,150,0,11,92,118,77,71,...,36,90,593,194,579,532,4873,3045,1494,294


In [99]:
get_selected_attributes_names(edu_data_df, metadata_replacement_dict)

['id2',
 'est__total',
 'est__male',
 'est__male___no_schooling_completed',
 'est__male___nursery_to_4th_grade',
 'est__male___5th_and_6th_grade',
 'est__male___7th_and_8th_grade',
 'est__male___9th_grade',
 'est__male___10th_grade',
 'est__male___11th_grade',
 'est__male___12th_grade,_no_diploma',
 'est__male___high_school_graduate_(includes_equivalency)',
 'est__male___some_college,_<1_year',
 'est__male___some_college,_1+_years,_no_degree',
 "est__male___associate's_degree",
 "est__male___bachelor's_degree",
 "est__male___master's_degree",
 'est__male___professional_school_degree',
 'est__male___doctorate_degree',
 'est__female',
 'est__female___no_schooling_completed',
 'est__female___nursery_to_4th_grade',
 'est__female___5th_and_6th_grade',
 'est__female___7th_and_8th_grade',
 'est__female___9th_grade',
 'est__female___10th_grade',
 'est__female___11th_grade',
 'est__female___12th_grade,_no_diploma',
 'est__female___high_school_graduate_(includes_equivalency)',
 'est__female___so

### Income

Household Income (All)

In [16]:
string.ascii_uppercase[:9]

'ABCDEFGHI'

In [102]:
income_bucket = 'acs_income'
table_code = 'B19001'
income_data_df = load_data_df(income_bucket, table_code)
income_metadata_df = load_metadata_df(income_bucket, table_code)
print('Original df dimensions: {}'.format(income_data_df.shape))

income_data_df = keep_cols(income_metadata_df, income_data_df, ['estimate'])
print('After keeping only estimates cols df dimensions: {}'.format(income_data_df.shape))

Original df dimensions: (33121, 37)
After keeping only estimates cols df dimensions: (33121, 18)


In [108]:
income_data_df.head()

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,HD01_VD10,HD01_VD11,HD01_VD12,HD01_VD13,HD01_VD14,HD01_VD15,HD01_VD16,HD01_VD17
0,Id2,Estimate; Total:,"Estimate; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more"
1,00601,5818,2630,783,584,324,318,222,286,126,88,192,141,24,51,17,19,13
2,00602,12719,3889,2131,1445,919,800,694,443,573,241,612,342,380,159,22,69,0
3,00603,19009,6211,2554,1908,1501,1113,1085,701,798,372,782,824,532,340,103,72,113
4,00606,1959,831,224,237,158,165,94,83,50,48,59,10,0,0,0,0,0


In [103]:
show_manhattan(income_data_df)
#ignore if it shows the row 0 and jacked up in Github, it doesn't do that on my local machine. 

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,HD01_VD10,HD01_VD11,HD01_VD12,HD01_VD13,HD01_VD14,HD01_VD15,HD01_VD16,HD01_VD17
0,Id2,Estimate; Total:,"Estimate; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more"
2559,10001,12347,1122,555,610,425,158,245,323,371,320,571,854,1230,1106,832,864,2761
2560,10002,33461,5025,4218,2314,2057,1467,1546,1127,1041,1216,1712,2475,2639,1894,1184,1399,2147
2561,10003,26699,1433,644,771,755,682,749,703,805,760,1193,1960,2316,2066,1786,2917,7159
2562,10004,1587,47,0,23,18,0,20,70,48,3,45,37,129,232,171,220,524
2563,10005,4239,290,66,73,0,60,0,52,3,32,114,209,212,625,342,570,1591
2564,10006,1815,72,72,0,11,0,0,0,19,0,107,102,130,246,184,190,682
2565,10007,2761,64,52,0,0,63,0,15,58,14,33,34,193,131,213,296,1595
2566,10009,29725,3879,2348,1275,1496,734,963,692,857,997,1559,1754,2903,3487,1546,1839,3396
2567,10010,15972,833,609,343,374,337,361,212,383,159,792,771,1673,1749,1384,1177,4815


Household Income (whites alone)

In [104]:
income_bucket = 'acs_income'
table_code = 'B19001A'
income_whites_data_df = load_data_df(income_bucket, table_code)
income_whites_metadata_df = load_metadata_df(income_bucket, table_code)
print('Original df dimensions: {}'.format(income_whites_data_df.shape))

income_whites_data_df = keep_cols(income_whites_metadata_df, income_whites_data_df, ['estimate'])
print('After keeping only estimates cols df dimensions: {}'.format(income_whites_data_df.shape))

Original df dimensions: (33121, 37)
After keeping only estimates cols df dimensions: (33121, 18)


In [105]:
show_manhattan(income_whites_data_df)
#ignore if it shows the row 0 and jacked up in Github, it doesn't do that on my local machine. 

,GEO.id2,HD01_VD01,HD01_VD02,HD01_VD03,HD01_VD04,HD01_VD05,HD01_VD06,HD01_VD07,HD01_VD08,HD01_VD09,HD01_VD10,HD01_VD11,HD01_VD12,HD01_VD13,HD01_VD14,HD01_VD15,HD01_VD16,HD01_VD17
0,Id2,Estimate; Total:,"Estimate; Total: - Less than $10,000","Estimate; Total: - $10,000 to $14,999","Estimate; Total: - $15,000 to $19,999","Estimate; Total: - $20,000 to $24,999","Estimate; Total: - $25,000 to $29,999","Estimate; Total: - $30,000 to $34,999","Estimate; Total: - $35,000 to $39,999","Estimate; Total: - $40,000 to $44,999","Estimate; Total: - $45,000 to $49,999","Estimate; Total: - $50,000 to $59,999","Estimate; Total: - $60,000 to $74,999","Estimate; Total: - $75,000 to $99,999","Estimate; Total: - $100,000 to $124,999","Estimate; Total: - $125,000 to $149,999","Estimate; Total: - $150,000 to $199,999","Estimate; Total: - $200,000 or more"
2559,10001,8253,543,219,404,239,98,160,133,161,120,370,553,944,891,547,620,2251
2560,10002,12220,1345,963,395,480,313,339,503,279,287,479,1008,1486,1016,852,862,1613
2561,10003,21992,1067,505,678,534,537,473,531,580,532,1089,1619,1896,1753,1397,2600,6201
2562,10004,1187,37,0,23,18,0,20,62,30,0,35,17,98,125,157,194,371
2563,10005,3103,107,28,73,0,60,0,20,3,14,44,168,104,486,188,445,1363
2564,10006,1264,28,63,0,11,0,0,0,0,0,46,74,55,149,128,171,539
2565,10007,2295,46,52,0,0,22,0,15,41,0,14,34,193,119,189,260,1310
2566,10009,19385,1637,918,532,778,469,534,346,635,763,857,1193,2297,2753,1197,1650,2826
2567,10010,12347,470,505,225,256,328,216,187,344,159,613,584,1430,1197,1006,878,3949


In [106]:
get_selected_attributes_names(income_whites_data_df, metadata_replacement_dict)

['id2',
 'est__total',
 'est__total___<$10,000',
 'est__total___$10,000_to_$14,999',
 'est__total___$15,000_to_$19,999',
 'est__total___$20,000_to_$24,999',
 'est__total___$25,000_to_$29,999',
 'est__total___$30,000_to_$34,999',
 'est__total___$35,000_to_$39,999',
 'est__total___$40,000_to_$44,999',
 'est__total___$45,000_to_$49,999',
 'est__total___$50,000_to_$59,999',
 'est__total___$60,000_to_$74,999',
 'est__total___$75,000_to_$99,999',
 'est__total___$100,000_to_$124,999',
 'est__total___$125,000_to_$149,999',
 'est__total___$150,000_to_$199,999',
 'est__total___$200,000+']